In [3]:
import os
import re
import numpy as np 
import pandas as pd
import math

from pymatgen.core import Structure, PeriodicSite

import matplotlib.pyplot as plt
import seaborn as sns

# Define structure

In [4]:
structure_list = ['ABAVIJ_clean']

In [5]:
for structure in structure_list:
    structure = structure 

In [6]:
def get_supercell(structure):
    with open("nvt_results/%s/simulation.input" %structure, "r") as f_input:
        for line in f_input:
            if re.search("UnitCells", line):
                s1, s2, s3 = line.split()[1], line.split()[2], line.split()[3]
    supercell = [int(s1), int(s2), int(s3)]
    return supercell

In [7]:
sc = get_supercell(structure)

# Lattice from different output

In [8]:
# input cif, unit cell
cry_cif = Structure.from_file("nvt_results/%s/%s.cif" %(structure, structure))

In [9]:
# output cif, supercell
sp_cry = Structure.from_file("nvt_results/%s/Movies/System_0/Framework_0_final_%s_%s_%s_P1.cif" %(structure, sc[0], sc[1], sc[2]))

/home/xiaoqi/anaconda/lib/python3.9/site-packages/pymatgen/io/cif.py:669: UserWarning: A 1-line symmetry op P1 CIF is detected!
  warnings.warn(msg)
/home/xiaoqi/anaconda/lib/python3.9/site-packages/pymatgen/io/cif.py:1155: UserWarning: Issues encountered while parsing CIF: A 1-line symmetry op P1 CIF is detected!
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


In [10]:
# output vasp, supercell 
final_cry =  Structure.from_file("nvt_results/%s/Movies/System_0/Framework_0_final.vasp" %structure)

## Comparison

In [11]:
cry_cif.lattice

Lattice
    abc : 9.806399999999998 10.6685 13.9712
 angles : 95.30240000000002 110.54600000000002 74.7329
 volume : 1320.3832557017195
      A : 9.182621985018018 0.0 -3.441647053412613
      B : 2.630535822739987 10.291995607170083 -0.9859005818404675
      C : 0.0 0.0 13.9712

In [12]:
sp_cry.lattice

Lattice
    abc : 29.4192 32.0055 27.9424
 angles : 95.30240000000002 110.546 74.7329
 volume : 23766.89860263095
      A : 27.547865955054057 0.0 -10.32494116023784
      B : 7.891607468219961 30.875986821510246 -2.9577017455214025
      C : 0.0 0.0 27.9424

In [13]:
final_cry.lattice

Lattice
    abc : 29.4192 32.0055 27.942399999985504
 angles : 95.30240000000276 110.54600000001115 74.7329
 volume : 23766.898602616893
      A : 29.4192 0.0 0.0
      B : 8.427660888007434 30.875986821585958 0.0
      C : -9.8066445000486 5.79428151047e-05 26.1650041354

- ABC(cry_cif) == ABC(sp_cry)/sc.
- ABC(sp_cry) != ABC(final_cry). 

- ABC(final_cry) is from Movies/System_0/Framework_0_final.vasp,  \
    """ \
    ABAVIJ_clean  \
    1.000000    \
    29.4192000000000000   0.0000000000000000         0.0000000000   \
    8.4276608880074342  30.8759868215859576         0.0000000000    \
    -9.8066445000486002   0.0000579428151047        26.1650041354   \
    """

# Matrix effect on coordinates

In [14]:
sp_cry.sites[1]

PeriodicSite: Co (9.0437, 3.9516, 7.0598) [0.2916, 0.1280, 0.3740]

In [15]:
final_cry.sites[1]

PeriodicSite: Co (5.9907, 3.9516, 9.7847) [0.2916, 0.1280, 0.3740]

# Gas molecule positions

In [16]:
with open("nvt_results/%s/Movies/System_0/Movie_%s_%d.%d.%d_298.000000_0.000000_allcomponents.pdb" %(structure, structure, sc[0], sc[1], sc[2])) as file:
    data = file.readlines()
    atom_pos = np.array([np.array([float(i) for i in line.split()[4:7]]) for line in data if "ATOM" in line])

In [17]:
# This is Cartesian coordinates
atom_pos[0]

array([18.475, 14.052,  3.122])

# Atoms in sphere step by step

## cif lattice

In [18]:
cry_cif.get_sites_in_sphere(pt=atom_pos[5], r=2)

[PeriodicSite: O (22.6933, 25.7629, 3.1170) [1.7542, 2.5032, 0.8319],
 PeriodicSite: C (21.6487, 26.0748, 3.7309) [1.6318, 2.5335, 0.8478],
 PeriodicSite: Co (23.4874, 24.5356, 1.6463) [1.8749, 2.3840, 0.7479]]

# vasp lattice

In [23]:
final_cry.get_sites_in_sphere(pt=atom_pos[5], r=3.5)

[PeriodicSite: H (22.5765, 26.8817, 5.4319) [0.5872, 0.8706, 0.2076]]

In [24]:
final_cry.get_sites_in_sphere(pt=atom_pos[5], r=3.7)

[PeriodicSite: H (22.5765, 26.8817, 5.4319) [0.5872, 0.8706, 0.2076],
 PeriodicSite: H (25.8880, 25.5519, 4.8458) [0.7046, 0.8276, 0.1852]]

In [25]:
final_cry.get_sites_in_sphere(pt=atom_pos[5], r=3.9)

[PeriodicSite: C (22.2040, 21.7696, 0.7549) [0.5624, 0.7051, 0.0289],
 PeriodicSite: C (22.1464, 28.0004, 0.7899) [0.5031, 0.9069, 0.0302],
 PeriodicSite: O (21.2751, 28.2968, 3.7398) [0.5083, 0.9165, 0.1429],
 PeriodicSite: C (21.8496, 26.6974, 6.0143) [0.5716, 0.8647, 0.2299],
 PeriodicSite: H (22.5765, 26.8817, 5.4319) [0.5872, 0.8706, 0.2076],
 PeriodicSite: N (23.1715, 28.4543, 1.5317) [0.5431, 0.9216, 0.0585],
 PeriodicSite: H (25.8880, 25.5519, 4.8458) [0.7046, 0.8276, 0.1852]]